In [22]:
import numpy as np
import pandas as pd

from itertools import product
from collections import Counter
from datasets import load_dataset
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score

In [2]:
def metric_calculation(pred, gt):    
    acc=accuracy_score(gt, pred)
    f1=f1_score(gt, pred, average='macro')
    confusion=confusion_matrix(gt, pred)
    fpr=confusion[0,1]/len(gt) ## predict to be 1; actual 0
    fnr=confusion[1,0]/len(gt) ## predict to be 0; actual 1
    return acc, f1, fpr, fnr
    
def post_processing(pred):
    new_pred=[]
    for i in pred:
        i=i.lower()
        if 'response' in i:
            try: new_pred.append(i.split('response')[1].split()[1].replace('</s>', ''))
            except: new_pred.append(2)
        elif 'output' in i:
            try: new_pred.append(i.split('output')[1].split()[1].replace('</s>', ''))
            except: new_pred.append(2)
        else:
            try: new_pred.append(i.split()[0].replace('</s>', ''))
            except:new_pred.append(2)
    new_pred = np.array([int(float(i)) if i in ['0', '0.0', '1', '1.0'] else 2 for i in new_pred])
    return new_pred

In [23]:
ds = load_dataset("beanham/spatial_join_dataset")
test=ds['test']
gt=np.array(test['label'])

In [28]:
results=[]
metric_values = ['worst_single', 'best_single', 'worst_comb']#, 'best_comb', 'worst_all', 'best_all']
for model in ['4o_mini', 'qwen']:
    for value in metric_values:
        zero_shot=np.load(f'base/{model}_correction/{model}_{value}_zero_shot_with_heur_value_all_correction.npy')
        #few_shot=np.load(f'base/{model}_correction/{model}_{value}_few_shot_with_heur_value_all_correction_reviews.npy')
        zero_shot=post_processing(zero_shot)
        #few_shot=post_processing(few_shot)
        zero_metrics=metric_calculation(zero_shot, gt)
        #few_metrics=metric_calculation(few_show, gt)
        results.append([model, value, 'zero_shot', zero_metrics[0], zero_metrics[1]])
        #results.append([model, value, 'few_shot', zero_metrics[0], zero_metrics[1]])
results=pd.DataFrame(results, columns=['model', 'value', 'prompt', 'acc', 'f1'])        

In [29]:
results

,model,value,prompt,acc,f1
0,4o_mini,worst_single,zero_shot,0.858,0.846106
1,4o_mini,best_single,zero_shot,0.936,0.931733
2,4o_mini,worst_comb,zero_shot,0.924,0.920281
3,qwen,worst_single,zero_shot,0.855,0.853057
4,qwen,best_single,zero_shot,0.921,0.917463
5,qwen,worst_comb,zero_shot,0.838,0.836849
